# Hermes — Run a basic Hartree-Fock energy calculation

See [the tutorial](/Tutorials/hermes_hartree_fock_energy_calculation.ipynb) for an in-depth explanation of how the Hermes Hartree-Fock energy calculation works.

In [ ]:
# |hide
import os
import pathlib

WORK_DIR = pathlib.Path("~/qdx/hermes_hartree_fock_quickstart/").expanduser()
if WORK_DIR.exists():
    !rm -r $WORK_DIR
os.makedirs(WORK_DIR)
os.chdir(WORK_DIR)

In [ ]:
# Get a pdb to work with
# We use the pdb-tools cli here, but you can download directly from rcsb.org
!pdb_fetch '1brs' | pdb_selchain -A | pdb_delhetatm > '1B39_A_nohet.pdb'
!ls

1B39_A_nohet.pdb


In [ ]:
import json
from pathlib import Path

import rush

client = rush.build_blocking_provider_with_functions(
    batch_tags=["hermes_quickstart"],
)

prepared_protein_qdxf, prepared_protein_pdb = client.prepare_protein(
    client.workspace / "1B39_A_nohet.pdb", None, None
)

# There may be multiple conformers, so select the first one
(first_conformer,) = client.pick_conformer(prepared_protein_qdxf, 0)

# Fragment the protein so that we can run a quantum energy calculation
(fragmented_protein,) = client.fragment_aa(first_conformer, 1, "All", 30)

fragmented_protein_contents = json.loads(
    fragmented_protein.download().read_text()
)

# Quantum energy calculation
(hermes_energy,) = client.hermes_energy(
    fragmented_protein,
    {},
    {
        "basis": "STO-3G",
        "aux_basis": "6-31G",
        "method": "RestrictedHF",
    },  # configuration for a fast converging, low accuracy run
    None,
    {
        "level": "Dimer",
        "cutoffs": {"dimer": 15},
        "reference_fragment": len(
            fragmented_protein_contents["topology"]["fragments"]
        )
        - 1,
    },
    None,
    resources={"gpus": 1, "storage": 100, "storage_units": "MB", "walltime": 60},
)

energy = json.load(open(hermes_energy.download(), "r"))
# Let's look at the expanded Hartree Fock energy
energy["expanded_hf_energy"]

2024-04-10 15:48:19,452 - rush - INFO - Not restoring by default via default
2024-04-10 15:48:22,228 - rush - INFO - Argument 3d0e45e8-af50-4a37-bdb3-9de0c3cb0110 is now ModuleInstanceStatus.RESOLVING
2024-04-10 15:49:11,312 - rush - INFO - Argument 3d0e45e8-af50-4a37-bdb3-9de0c3cb0110 is now ModuleInstanceStatus.ADMITTED
2024-04-10 15:49:14,572 - rush - INFO - Argument 3d0e45e8-af50-4a37-bdb3-9de0c3cb0110 is now ModuleInstanceStatus.DISPATCHED
2024-04-10 15:49:15,666 - rush - INFO - Argument 3d0e45e8-af50-4a37-bdb3-9de0c3cb0110 is now ModuleInstanceStatus.AWAITING_UPLOAD
2024-04-10 15:49:31,258 - rush - INFO - Argument 607a0a9f-17c4-4fa4-bb6b-59fd8048116d is now ModuleInstanceStatus.RESOLVING
2024-04-10 15:49:36,729 - rush - INFO - Argument 607a0a9f-17c4-4fa4-bb6b-59fd8048116d is now ModuleInstanceStatus.ADMITTED
2024-04-10 15:49:38,919 - rush - INFO - Argument 607a0a9f-17c4-4fa4-bb6b-59fd8048116d is now ModuleInstanceStatus.DISPATCHED
2024-04-10 15:49:40,058 - rush - INFO - Argument 

0.5647421741541621